In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Load dataset
df = pd.read_csv('../data/vendor_payments.csv', parse_dates=['Invoice_Date', 'Payment_Due_Date', 'Payment_Date'])
df.head()

In [ ]:
# Create delay columns
df['Delay_Days'] = (df['Payment_Date'] - df['Payment_Due_Date']).dt.days
df['is_delayed'] = df['Delay_Days'].apply(lambda x: 1 if x > 0 else 0)
df.head()

In [ ]:
# Visualizing delay distribution
plt.figure(figsize=(8,4))
sns.histplot(df['Delay_Days'], kde=True)
plt.title('Payment Delay Distribution')
plt.show()

# Department wise average delay
plt.figure(figsize=(8,4))
sns.boxplot(x='Department', y='Delay_Days', data=df)
plt.title('Department wise Delay')
plt.show()

In [ ]:
# Convert categorical columns
df_model = pd.get_dummies(df, columns=['Payment_Method', 'Department'])

# Features and target
X = df_model.drop(columns=['Invoice_ID', 'Vendor_ID', 'Invoice_Date', 'Payment_Due_Date', 'Payment_Date', 'Delay_Days', 'is_delayed'])
y = df_model['Delay_Days']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Model
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Predict and Evaluate
y_pred = model.predict(X_test)
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("R2 Score:", r2_score(y_test, y_pred))

In [ ]:
# Using Isolation Forest to detect anomalies
clf = IsolationForest(contamination=0.2, random_state=42)
df['anomaly_score'] = clf.fit_predict(df[['Delay_Days']])

# Show anomalies
anomalies = df[df['anomaly_score'] == -1]
print("Anomalies Detected:")
anomalies[['Invoice_ID', 'Delay_Days', 'Department']]